In [27]:
import json
import collections
import numpy as np
import scipy.stats

model1_name = "albert-large-v2"
model2_name = "bert-base-cased"
rte_model_address = "./experiments/rte/{}/supervised/"
setup_name = "32shots-p{}-i0-seed42/"
num_prompts = 27

models = ["roberta-large", "roberta-base", "deberta-v3-base", "deberta-v3-large", "deberta-v2-xlarge", "albert-base-v2", "albert-large-v2", "bert-base-uncased", "bert-base-cased", "xlnet-base-cased", "xlnet-large-cased"] #, "T5-base", "T5-11B"
model_combs = []
for i in range(len(models)):
    for j in range(i+1, len(models)):
        model_combs.append((models[i], models[j]))

In [28]:
def read_logits(address: str):
    logits_true, logits_false = [], []
    with open(address) as f:
        for line in f:
            logits = line.strip('\n').split()
            logits_true.append(float(logits[0]))
            logits_false.append(float(logits[1]))
    return logits_true, logits_false

def read_predictions(address: str, logits_true, logits_false):
    ln_idx_pred = dict()
    ln = 0
    with open(address) as f:
        for line in f:
            ln_dict = json.loads(line.strip('\n'))
            # if ln_dict['idx'] in ln_idx_pred.keys():
            #     print("fuck!")
            if ln_dict['label'] == "entailment":
                label = 1
            elif ln_dict['label'] == "not_entailment":
                label = -1
            ln_idx_pred[ln_dict['idx']] = (logits_true[ln], logits_false[ln], label)
            ln += 1
    return ln_idx_pred

def get_logits_each_line(split1_dict, split2_dict):
    split1_t, split2_t, split1_f, split2_f, split1_label, split2_label = [], [], [], [], [], []
    for idx, (t, f, label) in split1_dict.items():
        split1_t.append(t)
        split1_f.append(f)
        split1_label.append(label)
        split2_t.append(split2_dict[idx][0])
        split2_f.append(split2_dict[idx][1])
        split2_label.append(split2_dict[idx][2])

    return split1_t, split2_t, split1_f, split2_f, split1_label, split2_label

In [29]:
counter = []
for model1_name, model2_name in model_combs:
    model1, model2, model1_true, model1_false, model2_true, model2_false = [], [], [], [], [], []

    for i in range(num_prompts):
        dev1_true, dev1_false = read_logits(rte_model_address.format(model1_name) + setup_name.format(i) + "dev_eval_logits.txt")
        dev1_ln_tuple_dict = read_predictions(rte_model_address.format(model1_name) + setup_name.format(i) + "dev_predictions.jsonl", dev1_true, dev1_false)
        test1_true, test1_false = read_logits(rte_model_address.format(model1_name) + setup_name.format(i) + "test_eval_logits.txt")
        test1_ln_tuple_dict = read_predictions(rte_model_address.format(model1_name) + setup_name.format(i) + "test_predictions.jsonl", test1_true, test1_false)

        dev2_true, dev2_false = read_logits(rte_model_address.format(model2_name) + setup_name.format(i) + "dev_eval_logits.txt")
        dev2_ln_tuple_dict = read_predictions(rte_model_address.format(model2_name) + setup_name.format(i) + "dev_predictions.jsonl", dev2_true, dev2_false)
        test2_true, test2_false = read_logits(rte_model_address.format(model2_name) + setup_name.format(i) + "test_eval_logits.txt")
        test2_ln_tuple_dict = read_predictions(rte_model_address.format(model2_name) + setup_name.format(i) + "test_predictions.jsonl", test2_true, test2_false)

        dev1_t_line, dev2_t_line, dev1_f_line, dev2_f_line, dev1_label, dev2_label = get_logits_each_line(dev1_ln_tuple_dict, dev2_ln_tuple_dict)
        test1_t_line, test2_t_line, test1_f_line, test2_f_line, test1_label, test2_label = get_logits_each_line(test1_ln_tuple_dict, test2_ln_tuple_dict)

        model1.extend(dev1_t_line)
        model1.extend(dev1_f_line)
        model1.extend(test1_t_line)
        model1.extend(test1_f_line)
        model2.extend(dev2_t_line)
        model2.extend(dev2_f_line)
        model2.extend(test2_t_line)
        model2.extend(test2_f_line)
        model1_true.extend(dev1_t_line)
        model1_true.extend(test1_t_line)
        model1_false.extend(dev1_f_line)
        model1_false.extend(test1_f_line)
        model2_true.extend(dev2_t_line)
        model2_true.extend(test2_t_line)
        model2_false.extend(dev2_f_line)
        model2_false.extend(test2_f_line)
        if dev1_ln_tuple_dict.keys() == dev2_ln_tuple_dict.keys():
            print("fuck")
            counter.append(model1_name)
            counter.append(model2_name)
print(len(counter))

fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck
fuck


In [24]:
# Dev and test sets are fixed across different GPUs by the seed=42, since idxs are the same (changed the read_predictions function)
dev1_ln_tuple_dict.keys() == dev2_ln_tuple_dict.keys()

True

In [16]:
# pearsonr
roberta_large_roberta_base_pearsonr = (scipy.stats.pearsonr(model1, model2), scipy.stats.pearsonr(model1_true, model2_true), scipy.stats.pearsonr(model1_false, model2_false))
roberta_large_roberta_base_pearsonr

((-0.00032186762742251454, 0.9736423216580231),
 (0.05664751839558531, 3.8778100412595496e-05),
 (-0.018214156828060105, 0.18615201905990386))

In [17]:
# spearmanr
roberta_large_roberta_base_spearmanr = (scipy.stats.spearmanr(model1, model2), scipy.stats.spearmanr(model1_true, model2_true), scipy.stats.spearmanr(model1_false, model2_false))
roberta_large_roberta_base_spearmanr

(SpearmanrResult(correlation=0.008429875480452508, pvalue=0.38684029598442005),
 SpearmanrResult(correlation=0.05232089920954464, pvalue=0.00014473983731727512),
 SpearmanrResult(correlation=-0.023337614794761805, pvalue=0.09026196050988942))

In [18]:
# kendalltau
roberta_large_roberta_base_kendalltau = (scipy.stats.kendalltau(model1, model2), scipy.stats.kendalltau(model1_true, model2_true), scipy.stats.kendalltau(model1_false, model2_false))
roberta_large_roberta_base_kendalltau

(KendalltauResult(correlation=0.006236634872527052, pvalue=0.336923336742174),
 KendalltauResult(correlation=0.0341043870146642, pvalue=0.00020524870499968607),
 KendalltauResult(correlation=-0.01615139642814313, pvalue=0.07871749786322164))

In [11]:
# pearsonr
roberta_large_roberta_base_pearsonr = (scipy.stats.pearsonr(dev1_t_line, dev2_t_line), scipy.stats.pearsonr(dev1_f_line, dev2_f_line), scipy.stats.pearsonr(test1_t_line, test2_t_line), scipy.stats.pearsonr(test1_f_line, test2_f_line), scipy.stats.pearsonr(dev1_label, dev2_label), scipy.stats.pearsonr(test1_label, test2_label))
roberta_large_roberta_base_pearsonr


((0.05330755743590386, 0.40133501773399416),
 (0.010967930078873223, 0.8630017770015118),
 (-0.06513469939694615, 0.28000733084193763),
 (-0.0420143049030549, 0.4861765472217482),
 (0.025104218529954175, 0.6928387784316244),
 (-0.05016675510486945, 0.4055842889356936))

In [ ]:
# spearmanr
roberta_large_roberta_base_spearmanr = (scipy.stats.spearmanr(dev1_t_line, dev2_t_line), scipy.stats.spearmanr(dev1_f_line, dev2_f_line), scipy.stats.spearmanr(test1_t_line, test2_t_line), scipy.stats.spearmanr(test1_f_line, test2_f_line), scipy.stats.spearmanr(dev1_label, dev2_label), scipy.stats.spearmanr(test1_label, test2_label))
roberta_large_roberta_base_spearmanr

In [ ]:
# kendalltau
roberta_large_roberta_base_kendalltau = (scipy.stats.kendalltau(dev1_t_line, dev2_t_line), scipy.stats.kendalltau(dev1_f_line, dev2_f_line), scipy.stats.kendalltau(test1_t_line, test2_t_line), scipy.stats.kendalltau(test1_f_line, test2_f_line), scipy.stats.kendalltau(dev1_label, dev2_label), scipy.stats.kendalltau(test1_label, test2_label))
roberta_large_roberta_base_kendalltau

In [ ]:
# Accuracy
def read_accuracy(address):
    with open(address + "results.json") as f:
        tmp = json.load(f)
    return tmp['dev_set_after_training']['acc'], tmp['test_set_after_training']['acc']

model1_dev_acc, model1_test_acc, model2_dev_acc, model2_test_acc = [], [], [], []
for i in range(num_prompts):
    dev1_acc, test1_acc = read_accuracy(rte_rbt_large + setup_name.format(i))
    dev2_acc, test2_acc = read_accuracy(rte_rbt_base + setup_name.format(i))
    model1_dev_acc.append(dev1_acc)
    model1_test_acc.append(test1_acc)
    model2_dev_acc.append(dev2_acc)
    model2_test_acc.append(test2_acc)


In [ ]:
# pearsonr
roberta_large_roberta_base_pearsonr = (scipy.stats.pearsonr(model1_dev_acc, model2_dev_acc), scipy.stats.pearsonr(model1_test_acc, model2_test_acc))
roberta_large_roberta_base_pearsonr

In [ ]:
# spearmanr
roberta_large_roberta_base_spearmanr = (scipy.stats.spearmanr(model1_dev_acc, model2_dev_acc), scipy.stats.spearmanr(model1_test_acc, model2_test_acc))
roberta_large_roberta_base_spearmanr

In [ ]:
# kendalltau
roberta_large_roberta_base_kendalltau = (scipy.stats.kendalltau(model1_dev_acc, model2_dev_acc), scipy.stats.kendalltau(model1_test_acc, model2_test_acc))
roberta_large_roberta_base_kendalltau